In [3]:
import re
from pdfminer.high_level import extract_pages, extract_text

text = extract_text("../PDFs/Adam Smith - Private Loan Borrower Form.pdf")
print(text)

PRIVATE LOAN BORROWER FORM
To be submitted by private loan borrowers after credit approval by lender

Borrower Last Name: Smith

Borrower First Name: Adam

Date of Birth: 11/19/1982

Application ID: 423982870190

Your lender’s name: XYZ Capital

Your lender’s website home page: xyzcapital.com

Loan amount approved by your lender (US dollars): $4200

Signature: I understand that this form… I further understand that I must complete the following steps before my loan will be ready
for disbursement.

Borrower Signature Date: 03/27/2023

NEXT STEPS:

1. Review your Application Disclosure to find general information about your loan. Online applicants are able to click on a link to read the

disclosure. Students who apply by phone receive the disclosure via U.S. mail.

2. Once your loan is credit-approved, follow your lender's instructions for the next steps in your loan application process. 3.
Self-Certification-Complete the Applicant Self-Certification Form. You will receive it from your le

In [4]:
# borrower_last_name_pattern = re.compile(r"Borrower Last Name")
# matches = borrower_last_name_pattern.findall(text)
# print(matches)

['Borrower Last Name']
